In [1]:
import os
from dotenv import load_dotenv

load_dotenv("config/.token.env")
load_dotenv("config/.box.env")
load_dotenv("config/.openai.env")

openai_key = os.getenv("OPENAI_API_KEY")

box_developer_token=os.getenv("BOX_DEVELOPER_TOKEN")
os.environ['BOX_DEVELOPER_TOKEN'] = box_developer_token
box_search_query=os.getenv("BOX_SEARCH_QUERY")

In [2]:
from langchain_community.tools.box.box_file_search import BoxFileSearchTool
from langchain_community.tools.box.box_text_rep import BoxTextRepTool

box_search = BoxFileSearchTool(box_developer_token=box_developer_token)
search_results = box_search._run(box_search_query)
print(search_results)

GDBS: query "EXT. 8TH STREET BETWEEN AVENUES C AND D - DAY" token 7W3pUU3YrlWqh7tI1Eos5k9G9MZ68Chg
GDBS: files ['1233039227512', '1169674971571']
[Document(metadata={'source': 'https://dl.boxcloud.com/api/2.0/internal_files/1233039227512/versions/1346280085912/representations/extracted_text/content/', 'title': 'FIVE_FEET_AND_RISING_by_Peter_Sollett_pdf'}, page_content='\n3/20/23, 5:31 PM FIVE FEET AND RISING by Peter Sollett\n\nwww.dailyscript.com/scripts/fivefeetandrising.html 1/25\n\nFIVE FEET AND RISING\n\nby\n\nPeter Sollett\n\nFADE IN:\n\nEXT. 8TH STREET BETWEEN AVENUES C AND D - DAY \n\nA group of dark-skinned girls wearing cheerleading outfits \nalign themselves in formation on the sidewalk. They begin to \ndance. No music can be heard. The sound of the girls\' bodies \nis our soundtrack. We hear their strained breathing, palms \nand sneaker bottoms pounding while they hum and count softly \nto themselves in an effort to keep the rhythm.\n\nSLO-MO: We explore the bodies of the d

In [3]:
tools = [box_search]

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", openai_api_key=openai_key)

model.max_tokens = 900

from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content=box_search_query)])
response.content

"A vibrant, gritty slice of New York's Lower East Side. Hipsters with their unusual hairstyles, tattoos and piercings saunter by, mixed in with older, long-time residents. Elderly men and women sit on the stoops of their old brownstones, watching the world go by.\n\nStreet vendors hawk their wares, the smell of fresh pretzels and hot dogs wafting through the air. Bodegas and trendy boutiques line the street, a strange mix of the old and the new. Graffiti, both beautiful and obscene, adorns the sides of buildings.\n\nTraffic is heavy but not oppressive. Yellow cabs, food delivery bikes, and skateboarders weave through the congestion. The cacophony of horns, music, and chatter is constant, a living, breathing soundtrack to the city’s pulse.\n\nAmidst all this, a YOUNG WOMAN navigates her way through the crowd, her face a mask of determination and focus."

In [5]:
model_with_tools = model.bind_tools(tools)

In [6]:
response = model_with_tools.invoke([HumanMessage(content="find a document called five feet and rising, and create a voiceover for a 30 second commercial")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'box_file_search', 'args': {'query': 'five feet and rising'}, 'id': 'call_dqVHLcojSeRriNlzjwEDR31M'}]


In [5]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [8]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='a6b50593-5c72-4bb7-a76a-3cd9091be89c'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ee780438-4ddd-407c-8476-b52820c9c7e9-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="find a document called five feet and rising, and create a voiceover for a 30 second commercial. The content of this commercial should entice a viewer to want to see the movie. The voice should be that of a professional voice over actor, and the tone should be excited.")]}
)
response["messages"]

[HumanMessage(content='find a document called five feet and rising, and create a voiceover for a 30 second commercial. The content of this commercial should entice a viewer to want to see the movie. The voice should be that of a professional voice over actor, and the tone should be excited.', id='ba1054e5-5c09-4fce-baa6-3f92a36a0b0a'),
 AIMessage(content='Sure, let\'s start by finding the document. I\'ll use the `box_file_search` function for this.\n\n```tsx\n{\n  "__arg1": "five feet and rising"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 133, 'total_tokens': 174}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f4d95951-c4fb-4caa-b687-b75c54949eea-0', usage_metadata={'input_tokens': 133, 'output_tokens': 41, 'total_tokens': 174})]

In [6]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content=f"find a document containing the {box_search_query}. If there are multiple documents returned, choose only the first one. Then create a script for a voiceover based on the document for a 30 second commercial. The content of this commercial should entice a viewer to want to see the movie. The voice should be that of a professional voice over actor, and the tone should be excited.")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pD7fM2cTVwey6t82wtKfLwzS', 'function': {'arguments': '{\n  "query": "EXT. 8TH STREET BETWEEN AVENUES C AND D - DAY"\n}', 'name': 'box_file_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 171, 'total_tokens': 201}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6193c178-60ff-4821-b2fe-c7688bbd2297-0', tool_calls=[{'name': 'box_file_search', 'args': {'query': 'EXT. 8TH STREET BETWEEN AVENUES C AND D - DAY'}, 'id': 'call_pD7fM2cTVwey6t82wtKfLwzS'}], usage_metadata={'input_tokens': 171, 'output_tokens': 30, 'total_tokens': 201})]}}
----
GDBS: query EXT. 8TH STREET BETWEEN AVENUES C AND D - DAY token 7W3pUU3YrlWqh7tI1Eos5k9G9MZ68Chg
GDBS: files ['1233039227512', '1169674971571']
{'tools': {'messages': [ToolMessage(content='[Document(metadata={\'source\': \'https://dl.boxcl